In [1]:
# Lab 11 MNIST and Deep learning CNN
import tensorflow as tf
tf.set_random_seed(777)  # reproducibility

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for

# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
tf.reset_default_graph()

class Model:

    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()

    def _build_net(self):
        with tf.variable_scope(self.name):
            # dropout (keep_prob) rate  0.7~0.5 on training, but should be 1
            # for testing
            self.training = tf.placeholder(tf.bool)

            # input place holders
            self.X = tf.placeholder(tf.float32, [None, 784])
            # img 28x28x1 (black/white)
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])

            # Convolutional Layer #1
            conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3, 3],
                                     padding="SAME", activation=tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                            padding="SAME", strides=2)
            dropout1 = tf.layers.dropout(inputs=pool1,
                                         rate=0.7, training=self.training)
            print(conv1)
            print(pool1)

            # Convolutional Layer #2
            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3, 3],
                                     padding="SAME", activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2],
                                            padding="SAME", strides=2)
            dropout2 = tf.layers.dropout(inputs=pool2,
                                         rate=0.7, training=self.training)
            print(conv2)
            print(pool2)

            # L4 FC 7x7x64 inputs -> 1/8 outputs
            flat = tf.reshape(dropout2, [-1, 7 * 7 * 64])
            print(flat)
            dense4 = tf.layers.dense(inputs=flat,
                                     units=392, activation=tf.nn.relu)

            # L5 Final FC 625 inputs -> 10 outputs
            self.logits = tf.layers.dense(inputs=dense4, units=10)

        # define cost/loss & optimizer
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate=learning_rate).minimize(self.cost)

        is_correct = tf.equal(
            tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

    def predict(self, x_test, training=False):
        return self.sess.run(self.logits, feed_dict={self.X: x_test, self.training: training})

    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy, feed_dict={self.X: x_test, self.Y: y_test, self.training: training})

    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={
            self.X: x_data, self.Y: y_data, self.training: training})

In [3]:
# initialize
sess = tf.Session()
m1 = Model(sess, "m1")

sess.run(tf.global_variables_initializer())

print('Learning Started!')

# train my model
for epoch in range(training_epochs):
    avg_cost = 0
    batch_num = int(mnist.train.num_examples / batch_size)

    for i in range(batch_num):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = m1.train(batch_xs, batch_ys)
        avg_cost += c / batch_num

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
batch_num = int(mnist.test.num_examples / batch_size)
avg_accuracy = 0
for i in range(batch_num):
    batch_xs, batch_ys = mnist.test.next_batch(batch_size)
    avg_accuracy += m1.get_accuracy(batch_xs, batch_ys)
avg_accuracy /= batch_num
print('Accuracy:', avg_accuracy)

Tensor("m1/conv2d/Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("m1/max_pooling2d/MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("m1/conv2d_1/Relu:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("m1/max_pooling2d_1/MaxPool:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("m1/Reshape_1:0", shape=(?, 3136), dtype=float32)
Learning Started!
Epoch: 0001 cost = 0.409751490
Epoch: 0002 cost = 0.163148679
Epoch: 0003 cost = 0.126957436
Epoch: 0004 cost = 0.105961505
Epoch: 0005 cost = 0.092674651
Epoch: 0006 cost = 0.084720876
Epoch: 0007 cost = 0.079489941
Epoch: 0008 cost = 0.075601938
Epoch: 0009 cost = 0.070429185
Epoch: 0010 cost = 0.065663159
Epoch: 0011 cost = 0.063572258
Epoch: 0012 cost = 0.060741420
Epoch: 0013 cost = 0.059801823
Epoch: 0014 cost = 0.057626598
Epoch: 0015 cost = 0.056220703
Learning Finished!
Accuracy: 0.9924000060558319
